参考ページ 
https://qiita.com/mrpepper/items/f0199b28f967912bec21
https://web.mit.edu/music21/

musescoreのインストール ubuntu版   

1. appimageをダウンロード  
https://musescore.org/ja/download
 

2. 実行権限を与える．これだけでも，appimageをダブルクリックすれば動くようになる．  
chmod u+x MuseScore*.AppImage

3. インストール   
./MuseScore*.AppImage install  

4. music21のusersettingsのパスにmusescoreのインストール先を指定する．music21インストール後一回すればよい．
import music21 as m2
usersettings = m2.environment.UserSettings()
usersettings.create()
usersettings['musescoreDirectPNGPath'] = '/hoge/.local/bin/MuseScore-3.6.2.548021370-x86_64.AppImage'  
usersettings['musicxmlPath'] = '/hoge/.local/bin/MuseScore-3.6.2.548021370-x86_64.AppImage'

In [1]:
import music21 as m2
import pickle as pkl
from config import *

In [10]:
with open("save/part2_for_musixxml.pkl","rb") as f:
    [bar_rhythm_list,bar_pitch_list] = pkl.load(f)
    
#with open("save/part2_for_musixxml.pkl","rb") as f:
#    [bar_rhythm_list1,bar_pitch_list1] = pkl.load(f)

In [11]:
bar_rhythm_list =  bar_rhythm_list * 2
bar_pitch_list =  bar_pitch_list * 2

In [12]:
ind = 0

In [13]:
# ストリーム全体に関わる設定：楽器名とか調とか
mystrm = m2.stream.Part()
# 楽器
myinst = m2.instrument.Instrument()
myinst.partName = "Kiritan"
mystrm.append(myinst)
# 調号
mystrm.append(m2.key.Key('c', 'major'))
# 音部記号
mystrm.append(m2.clef.TrebleClef())
# mystrm.append(m2.clef.BassClef()) へ音記号

In [14]:
lyric = [i for i in "それってあなたのかんそうですよね"]
len_lyric = len(lyric)

In [15]:
ind_lyric = 0
for lengths,pitches in zip(bar_rhythm_list,bar_pitch_list): # 8小節
    print("---")
    cur_measure = m2.stream.Measure() 
    cur_measure.append( m2.tempo.MetronomeMark(number=120) )
    for length,pitch in zip(lengths,pitches): #1小節
        print(length,pitch)
        note,note_rest = length.split("_")
        quater_length = note2long_dicts[note]/24
        
        if note_rest == "note":
            #note2long_dicts
            note_number = key2array_dicts[pitch] - 12
            print(note_number)
            #if note_number >= 74:
#                note_number -= 12

            if note_number <= 71:
                note_number += 12
    
            char = lyric[ind_lyric%len_lyric]
            ind_lyric += 1
            n = m2.note.Note(note_number, quarterLength=quater_length, lyric=char)

        else:# 休符
            n = m2.note.Rest(quarterLength=quater_length)
            # 音符を放り込んだ小節をストリームに放り込む
        cur_measure.append(n)        
    mystrm.append(cur_measure)

---
quarter_note C4
60
quarter_note D4
62
quaver_note D4
62
quarter_note C4
60
quaver_note E5
76
---
quarter_note E5
76
quarter_note D5
74
quaver_note D5
74
quarter_note D5
74
quaver_note C4
60
---
quarter_note C4
60
quarter_note G4
67
quaver_note E5
76
quarter_note C4
60
quaver_note E5
76
---
quarter_note E5
76
quarter_note C4
60
quarter_note D5
74
quarter_note C4
60
---
quarter_note C4
60
quarter_note D4
62
quaver_note D4
62
quarter_note C4
60
quaver_note E5
76
---
quarter_note E5
76
quarter_note D5
74
quaver_note D5
74
quarter_note D5
74
quaver_note C4
60
---
quarter_note C4
60
quarter_note G4
67
quaver_note E5
76
quarter_note C4
60
quaver_note E5
76
---
quarter_note E5
76
quarter_note C4
60
quarter_note D5
74
quarter_note C4
60


In [16]:
mystrm.write(fp="part2.musicxml")

'C:\\Users\\yusuke\\Desktop\\Projects\\ai-music\\part2.xml'

In [23]:
for x in range(8):
    # 小節を作って、そこに音符を放り込んでゆく
    cur_measure = m2.stream.Measure()
    #
    n = m2.note.Note(x + 64, quarterLength=0.5, lyric='よ')
    # コンマ型ブレスは、直前の音符に属している
    b = m2.articulations.BreathMark()
    b.symbol = 'comma'
    n.articulations.append(b)
    cur_measure.append(n)
    # 休符
    cur_measure.append(m2.note.Rest(quarterLength=2))
    # 音符を放り込んだ小節をストリームに放り込む
    mystrm.append(cur_measure)

# タイでつなぐ（始点ならstart）
cur_measure = m2.stream.Measure()
cur_measure.append(m2.note.Rest(quarterLength=2))
n = m2.note.Note(60, quarterLength=2, lyric='う')
n.tie = m2.tie.Tie('start')
cur_measure.append(n)
mystrm.append(cur_measure)

# タイでつなぐ（3つつなぎなら真ん中はcontinue）
cur_measure = m2.stream.Measure()
n = m2.note.Note(60, quarterLength=4, lyric='ー')
n.tie = m2.tie.Tie('continue')
cur_measure.append(n)
mystrm.append(cur_measure)

# タイでつなぐ（終点ならstop）
cur_measure = m2.stream.Measure()
n = m2.note.Note(60, quarterLength=2, lyric='ー')
n.tie = m2.tie.Tie('stop')
cur_measure.append(n)
cur_measure.append(m2.note.Rest(quarterLength=2))
mystrm.append(cur_measure)


In [ ]:
mystrm.write()

PosixPath('/home/hyusuke/Desktop/Projects/ai_music/ai-music/test.musicxml')